In [1]:
from modes import *
import numpy as np
import os
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, UpSampling2D

In [2]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(201, 201, 7)))
model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, kernel_size=(3, 3), strides=2, activation='relu'))
model.add(Conv2D(1, kernel_size=(3, 3), padding='same', activation='sigmoid'))
model.add(UpSampling2D(size=(2, 2)))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 201, 201, 64)      4096      
                                                                 
 conv2d_1 (Conv2D)           (None, 201, 201, 128)     73856     
                                                                 
 conv2d_2 (Conv2D)           (None, 201, 201, 128)     147584    
                                                                 
 conv2d_3 (Conv2D)           (None, 100, 100, 64)      73792     
                                                                 
 conv2d_4 (Conv2D)           (None, 100, 100, 1)       577       
                                                                 
 up_sampling2d (UpSampling2D  (None, 200, 200, 1)      0         
 )                                                               
                                                        

In [3]:
def load_data(bc_code, load_code, penal, r, vf):
    bc_code, load_code, penal, r, vf = [param.numpy() for param in (bc_code, load_code, penal, r, vf)]
    bc_code, load_code = bc_code.decode('utf-8'), load_code.decode('utf-8')
    load_path = 'data/' + bc_code + '/' + load_code + '/'
    code = f'{bc_code}-{load_code}-P{penal}-R{r:0.3f}-V{vf:0.3f}'

    c = np.load(load_path + bc_code + load_code + '-C.npy')
    ux = np.load(load_path + bc_code + load_code + '-X.npy')
    uy = np.load(load_path + bc_code + load_code + '-Y.npy')

    x = np.ones((201, 201, 7), dtype=np.float32)
    x[:, :, 0] = ux
    x[:, :, 1] = uy
    x[:, :, 2] = np.hypot(ux, uy)
    x[:, :, 3] = np.pad(c, pad_width=1, mode='edge')[0:-1, 0:-1]
    x[:, :, 4], x[:, :, 5], x[:, :, 6] = penal, r, vf
    y = np.load(load_path + 'output/' + code + '.npy').astype(np.float32)

    return x, y

In [4]:
def general_function(bc_code, load_code, penal, r, vf):
    x, y = tf.py_function(load_data, (bc_code, load_code, penal, r, vf), (tf.float32, tf.float32))
    x.set_shape(tf.TensorShape([201, 201, 7]))
    y.set_shape(tf.TensorShape([200, 200]))
    return x, y

In [5]:
parameters = [(b, l, p, r, v) for b in list(bcs.keys()) for l in list(loads.keys())
              for p in ps for r in rs for v in vfs]
np.random.shuffle(parameters)
parameters = tuple(np.array(item) for item in zip(*parameters))

In [6]:
def filter_function(bc_code, load_code, penal, r, vf):
    bc_code, load_code, penal, r, vf = [param.numpy() for param in (bc_code, load_code, penal, r, vf)]
    bc_code, load_code = bc_code.decode('utf-8'), load_code.decode('utf-8')
    load_path = 'data/' + bc_code + '/' + load_code + '/'
    code = f'{bc_code}-{load_code}-P{penal}-R{r:0.3f}-V{vf:0.3f}'
    y_path = load_path + 'output/' + code + '.npy'
    c_path = load_path + bc_code + load_code + '-C.npy'
    return os.path.exists(y_path) and np.load(c_path).any()

In [7]:
dataset = tf.data.Dataset.from_tensor_slices(parameters)
dataset = dataset.filter(lambda b, l, p, r, v: tf.py_function(filter_function, (b, l, p, r, v), tf.bool))
dataset = dataset.map(lambda b, l, p, r, v: general_function(b, l, p, r, v))
dataset = dataset.batch(25)

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.fit(dataset, epochs=1, batch_size=25)

   4355/Unknown - 59816s 14s/step - loss: 0.1993 - accuracy: 0.6207